In [ ]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [ ]:
def get_script(fname:str):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),str(x))
    return 'try{' + f'console.log(`{fname}`,JSON.parse(`{json.dumps(args)}`));'+acc+'}catch(err){console.log(`{fname}`,err);}'
  return replacer
def script_runner(driver):
  def run_script(name:str,verbose=False):
    def script(*args:Any):
      script_string = get_script(name)(*args)
      answer = driver.execute_script(script_string)
      if verbose:
        print('script_string: ',script_string,' Answer: ', answer)
      return answer
    return script
  return run_script

In [ ]:
from Surfer import GetBrowser,RedditFeed,Feed

driver = GetBrowser()

In [ ]:
run_script = script_runner(driver)

In [ ]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [ ]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)


In [17]:
class RedditFeed(Feed):
	def __init__(self, driver,skip=True,verbose=False):
		super()
		self.name = 'Reddit Feed'
		# Store the WebDriver instance
		self.driver = driver
		self.verbose = verbose
		self.in_feed = True
		if not skip:
			self.driver.get('https://www.reddit.com/')
			run_script('listen to clicks')()

		run_script('init_reddit',verbose=self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True and self.in_feed:
			if not run_script('check curser',verbose=self.verbose)():
				continue
			run_script('scroll 2 curser',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			while run_script('check still scrolling')():
				time.sleep(0.1)
			yield run_script('parse post',verbose=self.verbose)()
			run_script('curser next')()

	def click_post(self, newTab=True):
		id = uuid()
		run_script('mark curser', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		# Get the dimensions and position of the element
		element_rect = run_script('curser location')()

		# Calculate the x and y coordinates of the random point in the upper center of the element
		X = element_rect['x'] + N(element_rect["width"] // 2,0.05) 
		Y = element_rect['y'] + random.randint(0, element_rect["height"] // 2)
			# Move the mouse to the calculated coordinates and click
		# actions = ActionChains(driver)
		element.click()
		self.in_feed = False
		# ic(X)
		# ic(Y)
		# if newTab:
		# 		actions.move_to_element_with_offset(element, X, Y) \
		# 				.key_down(Keys.CONTROL) \
		# 				.click() \
		# 				.key_up(Keys.CONTROL) \
		# 				.perform()
		# else:
		# 		actions.move_to_element_with_offset(element, X, Y) \
		# 				.click() \
		# 				.perform()


	def close(self):
		# Close the WebDriver instance
		self.driver.quit()

In [20]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [21]:
time.sleep(4)
for x in islice(rf,15):
  print(x)
  time.sleep(2*max(1,len(x) * 0.01))
  rf.click_post()

{'comments': '7.3k Comments', 'excerpt': None, 'links': [{'link': 'https://www.reddit.com/r/facepalm/', 'text': '7.3k Comments'}, {'link': 'https://www.reddit.com/r/facepalm/', 'text': ''}, {'link': 'https://www.reddit.com/user/abbiebe89/', 'text': ' 🇲\u200b🇮\u200b🇸\u200b🇨\u200b'}, {'link': 'https://www.reddit.com/r/facepalm/comments/12wqhvi/nashville_tennessee_christian_school_refused_to/', 'text': 'Nashville, Tennessee Christian School refused to allow a female student to enter prom because she was wearing a suit.'}, {'link': 'https://www.reddit.com/r/facepalm/?f=flair_name%3A%22%3AMisc%3A%20%F0%9F%87%B2%E2%80%8B%F0%9F%87%AE%E2%80%8B%F0%9F%87%B8%E2%80%8B%F0%9F%87%A8%E2%80%8B%22', 'text': 'u/abbiebe89'}, {'link': 'https://www.reddit.com/r/facepalm/comments/12wqhvi/nashville_tennessee_christian_school_refused_to/', 'text': 'r/facepalm'}, {'link': 'https://www.reddit.com/r/facepalm/comments/12wqhvi/nashville_tennessee_christian_school_refused_to/', 'text': ''}], 'rest': [' 🇲\u200b🇮\u200

ic| X: 333.06363869767233
ic| Y: 173


KeyboardInterrupt: 

In [13]:
rf.click_post()

ic| X: 332.95704261695
ic| Y: 234.21875


In [ ]:
# decultter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })